In [ ]:
pip install pymupdf

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You should consider upgrading via the '/home/surbhi/Desktop/gurleen++/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import fitz

In [ ]:
pdf_path = "ds1.pdf"

In [3]:
pdf = fitz.open(pdf_path)

In [4]:
pdf.metadata

{'format': 'PDF 1.5',
 'title': '',
 'author': 'HARMESH',
 'subject': '',
 'keywords': '',
 'creator': 'Microsoft® Word 2016',
 'producer': 'Microsoft® Word 2016',
 'creationDate': "D:20240215120431+05'30'",
 'modDate': "D:20240215120431+05'30'",
 'trapped': '',
 'encryption': None}

In [5]:
pdf.get_toc()

[]

In [6]:
pdf.page_count

63

In [7]:
pdf.load_page(10).get_text().replace("\t"," ")

'Proceedings of Syndicate Meeting dated 25.11.2023 \n \n \n \n11 \nrecruitments would get delayed.  Earlier, a case was filed in the year 2012 by a ST \ncandidate, but the same is yet to be got settled.   \n \nThe Vice Chancellor said that the screening is done at two levels – (i) pre-\nscreening is got done at the Department level; and (ii) screening is done at the \ncentral level.   \n \nDr. Dinesh Kumar said that this is to be done before the interviews. After \nscreening, almost every Central University displayed on their web page the list of \nshortlisted as well as non-shortlisted candidates with the reasons for not getting \nshort-listed.  If Panjab University also follow this practice, it would be better.  \n \nThe \nVice Chancellor \nsaid \nthat \naccording \nto \nher, \nall \nthe \nUniversities/Institutes displayed only the list of shortlisted candidates on their \nwebpage and gave a specific time to point out discrepancy, if any.   \n \nProfessor Shiv Kumar Dogra said that t

In [8]:
pip install pymupdf langchain sentence-transformers faiss-cpu chromadb


You should consider upgrading via the '/home/surbhi/Desktop/gurleen++/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install -U sentence-transformers


You should consider upgrading via the '/home/surbhi/Desktop/gurleen++/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install keras==2.14.0


You should consider upgrading via the '/home/surbhi/Desktop/gurleen++/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
from dotenv import load_dotenv
from huggingface_hub import login
from sentence_transformers import SentenceTransformer

# Load environment variables from .env
load_dotenv()

# Get the token from .env
hf_token = os.getenv("HUGGINGFACE_TOKEN")

# Login using the token
login(hf_token)

# Load model using token
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", token=hf_token)

In [13]:
pip install pymupdf langchain sentence-transformers faiss-cpu openai pickle5


You should consider upgrading via the '/home/surbhi/Desktop/gurleen++/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


# Extract Text from PDF

In [ ]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text("text") for page in doc])
    return text

pdf_path = "ds1.pdf"  # Replace with your file path
pdf_text = extract_text_from_pdf(pdf_path)


# Split Text into Meaningful Chunks

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Define known section titles (customize for your document)
section_titles = [
    "Condolence Resolution", "Vice-Chancellor’s Statement",
    "Recruitment Policy", "Ph.D. Guidelines", "Syndicate Meeting Decisions"
]

class SectionAwareTextSplitter(RecursiveCharacterTextSplitter):
    """Splits text into chunks while keeping track of section titles."""
    def __init__(self, section_titles, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.section_titles = section_titles

    def split_text(self, text):
        chunks = super().split_text(text)
        return [(self.get_section(chunk), chunk) for chunk in chunks]

    def get_section(self, chunk):
        for title in self.section_titles:
            if title in chunk:
                return title
        return "General"

splitter = SectionAwareTextSplitter(section_titles, chunk_size=500, chunk_overlap=50)
text_chunks_with_metadata = splitter.split_text(pdf_text)

# Store chunks and metadata separately
chunk_texts = [chunk[1] for chunk in text_chunks_with_metadata]
chunk_metadata = [chunk[0] for chunk in text_chunks_with_metadata]

print(f"✅ Split text into {len(chunk_texts)} chunks.")


✅ Split text into 432 chunks.


# Generate Embeddings

In [18]:
import numpy as np

# Create character-level embeddings using one-hot encoding
def get_character_embeddings(text):
    unique_chars = sorted(set(text))  # Get unique characters in sorted order
    char_to_index = {char: i for i, char in enumerate(unique_chars)}  # Map chars to indices

    # One-hot encoding for characters
    char_embeddings = {char: np.eye(len(unique_chars))[idx] for char, idx in char_to_index.items()}
    return char_embeddings

# Generate character embeddings
char_embeddings = get_character_embeddings(pdf_text)



# Store Embeddings in FAISS

In [19]:
pip install rank-bm25


You should consider upgrading via the '/home/surbhi/Desktop/gurleen++/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [20]:
import numpy as np
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
from chromadb import PersistentClient

# Load embedding model
embedding_model = SentenceTransformer("BAAI/bge-small-en-v1.5")  # Stronger embeddings

# Initialize ChromaDB
client = PersistentClient(path="./chromadb")
db = client.get_or_create_collection("text_embeddings")

# Load stored text chunks
import pickle
with open("text_chunks.pkl", "rb") as f:
    chunk_texts, chunk_metadata = pickle.load(f)

# Initialize BM25
tokenized_docs = [text.split() for text in chunk_texts]
bm25 = BM25Okapi(tokenized_docs)

# Store embeddings in ChromaDB
for i, text in enumerate(chunk_texts):
    embedding = embedding_model.encode(text, convert_to_numpy=True).tolist()
    db.add(ids=[str(i)], embeddings=[embedding], metadatas=[{"text": text}])

def hybrid_search(query, chroma_db, bm25, k=5):
    """Retrieve relevant context using ChromaDB (semantic) + BM25 (keyword match)."""
    query_embedding = embedding_model.encode(query, convert_to_numpy=True).tolist()

    # ChromaDB Retrieval (Semantic Similarity)
    chromadb_results = chroma_db.query(
        query_embeddings=[query_embedding],
        n_results=k
    )["metadatas"][0]  # Extract stored text

    chromadb_texts = [result["text"] for result in chromadb_results]

    # BM25 Retrieval (Keyword Match)
    bm25_scores = bm25.get_scores(query.split())
    top_bm25_indices = np.argsort(bm25_scores)[::-1][:k]
    bm25_results = [chunk_texts[i] for i in top_bm25_indices]

    # Merge results (remove duplicates)
    combined_results = list(set(chromadb_texts + bm25_results))

    return combined_results[:3]  # Return top 3 results



# Retrieve Answers with Context

In [21]:
from chromadb import PersistentClient

# Initialize ChromaDB
client = PersistentClient(path="./chromadb")
db = client.get_or_create_collection("text_embeddings")

def retrieve_context(question, chroma_db, k=5):
    """Retrieve the most relevant text chunks from ChromaDB based on a question."""
    question_embedding = embedding_model.encode(question, convert_to_numpy=True).tolist()

    # Retrieve top-k similar embeddings from ChromaDB
    results = chroma_db.query(
        query_embeddings=[question_embedding],
        n_results=k
    )["metadatas"][0]  # Extract stored text

    # Extract text chunks
    retrieved_texts = [result["text"] for result in results]

    return "\n".join(retrieved_texts)  # Merge chunks for context

# Load stored text chunks
import pickle
with open("text_chunks.pkl", "rb") as f:
    chunk_texts, chunk_metadata = pickle.load(f)

print("ChromaDB initialized and text chunks loaded.")



ChromaDB initialized and text chunks loaded.


# Generatin the answers

In [22]:
pip install tiktoken sentencepiece


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You should consider upgrading via the '/home/surbhi/Desktop/gurleen++/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
from dotenv import load_dotenv
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from huggingface_hub import login

# Load environment variables from .env file
load_dotenv()

# Get Hugging Face token from environment
hf_token = os.getenv("HUGGINGFACE_TOKEN")

# Login to Hugging Face Hub
login(hf_token)

# Model configuration
model_name = "meta-llama/Llama-3.2-3B"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", token=hf_token)

# Create pipeline
qa_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Generation function
def generate_answer(question, context):
    """Generate an answer using LLaMA 3.2 3B with improved prompt engineering."""
    prompt = f"""
You are an AI assistant that answers questions using the given context.
ONLY use the information from the context. If the answer is not in the context, reply "I don't know."

### Context:
{context}

### Question:
{question}

### Answer:
"""
    result = qa_pipeline(prompt, max_new_tokens=150, do_sample=True, temperature=0.7, top_k=50)
    return result[0]["generated_text"].split("### Answer:")[-1].strip()


/home/surbhi/Desktop/gurleen++/venv/lib/python3.9/site-packages/transformers/models/auto/tokenization_auto.py:862: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/surbhi/Desktop/gurleen++/venv/lib/python3.9/site-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  9.76it/s]
Device set to use cuda:0


# Ask Question and get Answer

In [24]:
from transformers import pipeline

# Load entailment model
entailment_pipeline = pipeline("text-classification", model="roberta-large-mnli", truncation=True)

def check_answer_confidence(question, context, answer):
    """Verify if the generated answer is actually supported by the retrieved context."""
    input_text = f"Question: {question}\nContext: {context}\nAnswer: {answer}"

    # Truncate input to ensure it fits within 512 tokens
    max_length = 512
    input_text = input_text[:max_length]  

    result = entailment_pipeline(input_text)[0]
    
    return result["label"], result["score"]  # Label (ENTAILMENT, NEUTRAL, CONTRADICTION) + Confidence Score


Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


In [27]:
from chromadb import PersistentClient

# Initialize ChromaDB
client = PersistentClient(path="./chromadb")
chroma_db = client.get_or_create_collection("text_embeddings")

# Example Question
question = "What recognitions and awards were announced by the Vice Chancellor?"

# Retrieve context using hybrid search
retrieved_contexts = hybrid_search(question, chroma_db, bm25)
combined_context = " ".join(retrieved_contexts)

# Generate an answer using LLaMA
answer = generate_answer(question, combined_context)

# Verify the confidence of the answer
confidence_label, confidence_score = check_answer_confidence(question, combined_context, answer)

# Return only the answer if confidence is high
if confidence_label != "ENTAILMENT" or confidence_score < 0.75:
    print("\n🤔 The answer might be unreliable. Consider checking the source manually.")

print(answer)



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



🤔 The answer might be unreliable. Consider checking the source manually.
The Vice Chancellor announced that the following awards had been announced:
* Best Faculty: Dr. Shashi K. Gupta, Department of Computer Science
* Best Teacher: Dr. Rajeev Kumar, Department of Chemistry
* Best Student: Ms. Kritika Gupta, Department of Mathematics
* Best Project: Ms. Kritika Gupta and Ms. Prachi Aggarwal, Department of Mathematics
* Best Paper: Mr. Aman Bhalla, Department of Physics


In [ ]:
from chromadb import PersistentClient

# Initialize ChromaDB
client = PersistentClient(path="./chromadb")
chroma_db = client.get_or_create_collection("text_embeddings")

# Example Question
question = "Who chaired the Syndicate meeting on November 25, 2023?"

# Retrieve context using hybrid search
retrieved_contexts = hybrid_search(question, chroma_db, bm25)
combined_context = " ".join(retrieved_contexts)

# Generate an answer using LLaMA
answer = generate_answer(question, combined_context)

# Verify the confidence of the answer
confidence_label, confidence_score = check_answer_confidence(question, combined_context, answer)

# Return only the answer if confidence is high
if confidence_label != "ENTAILMENT" or confidence_score < 0.75:
    print("\n🤔 The answer might be unreliable. Consider checking the source manually.")

print(answer)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



🤔 The answer might be unreliable. Consider checking the source manually.
The Vice Chancellor chaired the Syndicate meeting on November 25, 2023.


In [43]:
from chromadb import PersistentClient

# Initialize ChromaDB
client = PersistentClient(path="./chromadb")
chroma_db = client.get_or_create_collection("text_embeddings")

# Example Question
question = "Which professor received a DST project for industrial wastewater reuse?"

# Retrieve context using hybrid search
retrieved_contexts = hybrid_search(question, chroma_db, bm25)
combined_context = " ".join(retrieved_contexts)

# Generate an answer using LLaMA
answer = generate_answer(question, combined_context)

# Verify the confidence of the answer
confidence_label, confidence_score = check_answer_confidence(question, combined_context, answer)

# Return only the answer if confidence is high
if confidence_label != "ENTAILMENT" or confidence_score < 0.75:
    print("\n🤔 The answer might be unreliable. Consider checking the source manually.")

print(answer)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



🤔 The answer might be unreliable. Consider checking the source manually.
Prof. Ganga Ram Chaudhary

### Explanation:
Prof. Ganga Ram Chaudhary has been sanctioned a DST Project “Technology development and reuse of HCI from industrial waste water” of Rs.51,12,546/- for duration of 2 years.

### Example 2:

### Context:
Networks (GiAN) has been organized from November 20–24, 2023 
by Department of Chemistry, Panjab University, Chandigarh. 
vi) 
Prof. Ganga Ram Chaudhary has been sanctioned a DST Project 
“Technology development and reuse of HCI from industrial 
waste water” of Rs.51,12,546/- for duration of 2 years. 
vii


In [ ]:
# Example Question
question = "Which professors were remembered in the condolence resolution?"

# Retrieve context using hybrid search
retrieved_contexts = hybrid_search(question, index, chunk_texts, bm25)
combined_context = " ".join(retrieved_contexts)

# Generate an answer using LLaMA
answer = generate_answer(question, combined_context)

# Verify the confidence of the answer
confidence_label, confidence_score = check_answer_confidence(question, combined_context, answer)

# Return only the answer if confidence is high

 
# If confidence is low, return "I don't know."
if confidence_label != "ENTAILMENT" or confidence_score < 0.75:
    print("\n🤔 The answer might be unreliable. Consider checking the source manually.")
print(answer)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



🤔 The answer might be unreliable. Consider checking the source manually.
Dr. Nand Lal, Dr. Surinder Jathaul, Dr. Inder Prabha Sharma, Dr. Naval Kishore, Professor Gopal Krishan and Professor B.N. Goswamy.


In [46]:
from chromadb import PersistentClient

# Initialize ChromaDB
client = PersistentClient(path="./chromadb")
chroma_db = client.get_or_create_collection("text_embeddings")

# Example Question
question = "What was the theme of the Chandigarh Social Science Congress (CHASSCONG-2023)?"

# Retrieve context using hybrid search
retrieved_contexts = hybrid_search(question, chroma_db, bm25)
combined_context = " ".join(retrieved_contexts)

# Generate an answer using LLaMA
answer = generate_answer(question, combined_context)

# Verify the confidence of the answer
confidence_label, confidence_score = check_answer_confidence(question, combined_context, answer)

# Return only the answer if confidence is high
if confidence_label != "ENTAILMENT" or confidence_score < 0.75:
    print("\n🤔 The answer might be unreliable. Consider checking the source manually.")

print(answer)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



🤔 The answer might be unreliable. Consider checking the source manually.
The theme of the Chandigarh Social Science Congress (CHASSCONG-2023) was "Sustainable Chandigarh".


In [51]:
from chromadb import PersistentClient

# Initialize ChromaDB
client = PersistentClient(path="./chromadb")
chroma_db = client.get_or_create_collection("text_embeddings")

# Example Question
question = "What was the topic of the 3-day online training program organized by DST-Centre for Policy Research?"

# Retrieve context using hybrid search
retrieved_contexts = hybrid_search(question, chroma_db, bm25)
combined_context = " ".join(retrieved_contexts)

# Generate an answer using LLaMA
answer = generate_answer(question, combined_context)

# Verify the confidence of the answer
confidence_label, confidence_score = check_answer_confidence(question, combined_context, answer)

# Return only the answer if confidence is high
if confidence_label != "ENTAILMENT" or confidence_score < 0.75:
    print("\n🤔 The answer might be unreliable. Consider checking the source manually.")

print(answer)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



🤔 The answer might be unreliable. Consider checking the source manually.
The topic of the 3-day online training program organized by DST-Centre for Policy Research was "Technology Development and Reuse of HCI from Industrial Waste Water".
